# Access to Microphone Device

## Metadata


|     Metadata      |  Value  |
|:------------------|:---|
| collaborators     | ['@Cyb3rWard0g', '@Cyb3rPandaH'] |
| creation date     | 2020/06/09 |
| modification date | 2020/09/20 |
| playbook related  | [] |

## Hypothesis
Adversaries might be accessing the microphone in endpoints over the network.

## Technical Context
None

## Offensive Tradecraft
An adversary can leverage a computer's peripheral devices (e.g., microphones and webcams) or applications (e.g., voice and video call services) to capture audio recordings for the purpose of listening into sensitive conversations to gather information.
Based on some research from [@svch0st](https://twitter.com/svch0st) you can to determine when and how long a process had access to the microphone of an endpoint by monitoring the following registry key
  * HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Windows\CurrentVersion\CapabilityAccessManager\ConsentStore\microphone\.

## Security Datasets


| Metadata  |    Value  |
|:----------|:----------|
| docs      | https://securitydatasets.com/notebooks/atomic/windows/collection/SDWIN-200609225055.html        |
| link      | [https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/collection/host/msf_record_mic.zip](https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/collection/host/msf_record_mic.zip)  |

## Analytics

### Initialize Analytics Engine

In [ ]:
from openhunt.mordorutils import *
spark = get_spark()

### Download & Process Security Dataset

In [ ]:
sd_file = "https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/collection/host/msf_record_mic.zip"
registerSDSQLTable(spark, sd_file, "sdTable")

### Analytic I
Look for any creation or modification of registry keys under HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\CapabilityAccessManager\ConsentStore\microphone\NonPackaged


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| Windows Registry | Microsoft-Windows-Sysmon/Operational | Process created Windows registry key | 12 |
| Windows Registry | Microsoft-Windows-Sysmon/Operational | Process modified Windows registry key value | 13 |
| Windows Registry | Microsoft-Windows-Sysmon/Operational | Process modified Windows registry key value | 14 |
| Windows Registry | Microsoft-Windows-Sysmon/Operational | Process modified Windows registry key | 14 |

In [ ]:
df = spark.sql(
'''
SELECT EventID, Message
FROM sdTable
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
  AND EventID IN (12,13,14)
  AND LOWER(TargetObject) RLIKE '.*consentstore\\\\\\\microphone.*'
'''
)
df.show(10,False)

### Analytic II
Look for any creation or modification of registry keys under HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\CapabilityAccessManager\ConsentStore\microphone\NonPackaged


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| Windows Registry | Microsoft-Windows-Security-Auditing | Process accessed Windows registry key | 4663 |
| Windows Registry | Microsoft-Windows-Security-Auditing | User accessed Windows registry key | 4663 |
| Windows Registry | Microsoft-Windows-Security-Auditing | Process requested access Windows registry key | 4656 |
| Windows Registry | Microsoft-Windows-Security-Auditing | User requested access Windows registry key | 4656 |
| Windows Registry | Microsoft-Windows-Security-Auditing | Process modified Windows registry key value | 4657 |
| Windows Registry | Microsoft-Windows-Security-Auditing | User modified Windows registry key value | 4657 |

In [ ]:
df = spark.sql(
'''
SELECT EventID, Message
FROM sdTable
WHERE LOWER(Channel) = 'security'
  AND EventID IN (4656,4663,4657)
  AND LOWER(ObjectName) RLIKE '.*consentstore\\\\\\\microphone.*'
'''
)
df.show(10,False)

## Known Bypasses

## False Positives
None

## Hunter Notes
None

## Hunt Output

| Type | Link |
| :----| :----|
| Sigma Rule | [https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/sysmon_camera_microphone_access.yml](https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/sysmon_camera_microphone_access.yml) |
| Sigma Rule | [https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/win_camera_microphone_access.yml](https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/win_camera_microphone_access.yml) |

## References
* https://medium.com/@7a616368/can-you-track-processes-accessing-the-camera-and-microphone-7e6885b37072